## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Description,Date
0,0,Port Alfred,-33.5906,26.8910,56.50,56,0,11.45,ZA,clear sky,2021-06-06 03:13:54
1,1,Maine-Soroa,13.2114,12.0241,86.05,42,100,8.77,NE,overcast clouds,2021-06-06 03:17:08
2,2,Tias,28.9611,-13.6450,64.15,68,75,10.36,ES,broken clouds,2021-06-06 03:17:08
3,3,Mahajanga,-15.7167,46.3167,69.82,83,20,4.61,MG,few clouds,2021-06-06 03:17:09
4,4,Suhut,38.5311,30.5458,44.44,83,0,3.49,TR,clear sky,2021-06-06 03:08:05


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
custom_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
custom_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Description,Date
1,1,Maine-Soroa,13.2114,12.0241,86.05,42,100,8.77,NE,overcast clouds,2021-06-06 03:17:08
6,6,Akyab,20.1500,92.9000,82.09,84,90,14.20,MM,light rain,2021-06-06 03:16:03
7,7,Albany,42.6001,-73.9662,73.92,72,24,1.01,US,few clouds,2021-06-06 03:13:49
12,12,Padang,-0.9492,100.3543,84.33,60,81,2.39,ID,broken clouds,2021-06-06 03:12:47
13,13,Victoria,22.2855,114.1577,87.96,73,42,7.00,HK,scattered clouds,2021-06-06 03:14:01
14,14,Cherskiy,68.7500,161.3000,70.81,27,36,22.97,RU,scattered clouds,2021-06-06 03:17:09
17,17,Kapaa,22.0752,-159.3190,84.18,71,40,1.01,US,scattered clouds,2021-06-06 03:13:56
21,21,Bucerias,20.7500,-105.3333,82.98,78,75,2.28,MX,broken clouds,2021-06-06 03:13:34
23,23,Rikitea,-23.1203,-134.9692,76.10,81,69,12.80,PF,broken clouds,2021-06-06 03:13:52
28,28,Atuona,-9.8000,-139.0333,79.34,69,18,14.18,PF,few clouds,2021-06-06 03:13:51


In [6]:
# 4a. Determine if there are any empty rows.
custom_cities_df.count()

City_ID        339
City           339
Lat            339
Lng            339
Max Temp       339
Humidity       339
Cloudiness     339
Wind Speed     339
Country        337
Description    339
Date           339
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = custom_cities_df.dropna()

clean_df.count()

City_ID        337
City           337
Lat            337
Lng            337
Max Temp       337
Humidity       337
Cloudiness     337
Wind Speed     337
Country        337
Description    337
Date           337
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
1,Maine-Soroa,NE,86.05,overcast clouds,13.2114,12.0241,
6,Akyab,MM,82.09,light rain,20.1500,92.9000,
7,Albany,US,73.92,few clouds,42.6001,-73.9662,
12,Padang,ID,84.33,broken clouds,-0.9492,100.3543,
13,Victoria,HK,87.96,scattered clouds,22.2855,114.1577,
14,Cherskiy,RU,70.81,scattered clouds,68.7500,161.3000,
17,Kapaa,US,84.18,scattered clouds,22.0752,-159.3190,
21,Bucerias,MX,82.98,broken clouds,20.7500,-105.3333,
23,Rikitea,PF,76.10,broken clouds,-23.1203,-134.9692,
28,Atuona,PF,79.34,few clouds,-9.8000,-139.0333,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = { 
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
hotel_df.head(30)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
1,Maine-Soroa,NE,86.05,overcast clouds,13.2114,12.0241,Complexe
6,Akyab,MM,82.09,light rain,20.1500,92.9000,Hotel Memory
7,Albany,US,73.92,few clouds,42.6001,-73.9662,
12,Padang,ID,84.33,broken clouds,-0.9492,100.3543,Mercure Padang
13,Victoria,HK,87.96,scattered clouds,22.2855,114.1577,Mini Hotel Central
14,Cherskiy,RU,70.81,scattered clouds,68.7500,161.3000,Gostinitsa
17,Kapaa,US,84.18,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
21,Bucerias,MX,82.98,broken clouds,20.7500,-105.3333,Villa del Palmar Flamingos Beach Resort & Spa ...
23,Rikitea,PF,76.10,broken clouds,-23.1203,-134.9692,Pension Maro'i
28,Atuona,PF,79.34,few clouds,-9.8000,-139.0333,Villa Enata


In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

clean_hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
1,Maine-Soroa,NE,86.05,overcast clouds,13.2114,12.0241,Complexe
6,Akyab,MM,82.09,light rain,20.1500,92.9000,Hotel Memory
7,Albany,US,73.92,few clouds,42.6001,-73.9662,
12,Padang,ID,84.33,broken clouds,-0.9492,100.3543,Mercure Padang
13,Victoria,HK,87.96,scattered clouds,22.2855,114.1577,Mini Hotel Central
14,Cherskiy,RU,70.81,scattered clouds,68.7500,161.3000,Gostinitsa
17,Kapaa,US,84.18,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
21,Bucerias,MX,82.98,broken clouds,20.7500,-105.3333,Villa del Palmar Flamingos Beach Resort & Spa ...
23,Rikitea,PF,76.10,broken clouds,-23.1203,-134.9692,Pension Maro'i
28,Atuona,PF,79.34,few clouds,-9.8000,-139.0333,Villa Enata


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Description</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]

In [15]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig = gmaps.figure(layout={'width': '900', 'height': '900px'}) 
#heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
 #            max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='900px', width='900'))